In [2]:
seed = 0

wav_file = (
    "/home/marnix/thesis/data/OPENGLOT/RepositoryI/Vowel_I/I_breathy_280Hz.wav"
)

In [3]:
print(seed)

0


In [4]:
print(wav_file)

/home/marnix/thesis/data/OPENGLOT/RepositoryI/Vowel_I/I_breathy_280Hz.wav


In [ ]:
a = 1
b = 2

'/home/marnix/thesis/data/OPENGLOT/RepositoryI/Vowel_I/I_breathy_280Hz.wav'

In [ ]:
seed